In [ ]:
#experiment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [ ]:
data = pd.read_csv('../project/processed_data_with_timeindex.csv',index_col = [0])
data.index = pd.to_datetime(data.index)


In [ ]:
data = pd.read_csv('processed_data.csv',index_col = [0])
data.index = pd.to_datetime(data.index)


In [ ]:
# clustering by daily load pattern
# compute average daily pattern
daily_pattern = pd.DataFrame(columns = data.columns)
for i in data.columns:
    daily_pattern[i] = np.nanmean(np.array(data[i][:(data.shape[0] - data.shape[0]%(60*24))]).reshape(-1,24*60),axis = 0)

In [ ]:
# clustering by daily load pattern
# compute average daily pattern
daily_pattern = pd.DataFrame(columns = data.columns)
for i in data.columns:
    daily_pattern[i] = np.nanmean(np.array(data[i][:(data.shape[0] - data.shape[0]%(60*24))]).reshape(-1,24*60),axis = 0)

In [ ]:
daily_pattern = daily_pattern.loc[:,daily_pattern.columns[daily_pattern.std(0)!=0]]

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
# plot elbow plot
def elbow_plot(X,n_clusters = np.arange(1,40,2)):
    distortions = []
    for n in n_clusters:
        kmmodel = KMeans(n)
        kmmodel.fit(X.T)
        distortions.append(sum(np.min(cdist(X.T, kmmodel.cluster_centers_, 'euclidean'), axis=1)) / X.T.shape[0])
    plt.plot(n_clusters, distortions, 'bx-')
    plt.xticks(n_clusters)
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')

In [ ]:
import pickle
def labeled_plot(X,k=31,xnum=4,ynum=8,silent = True):
    '''
    k should <= xnum*ynum
    '''
    X = X.reset_index(drop = True)
    kmmodel = KMeans(k)
    kmmodel.fit(X.T)
    clusters_dict = {label:0 for label in set(kmmodel.labels_)}
    for i in kmmodel.labels_:
        clusters_dict[i] += 1
    count = pd.DataFrame(clusters_dict,index = ['count']).T.sort_values(by = 'count',ascending = False)
    clusters_dict = {label:0 for label in count.index}
    for label in count.index:
        clusters_dict[label] = X.iloc[:,kmmodel.labels_ == label]
    if not silent:
        fig = plt.figure(figsize=(20,8))
        for k,label in enumerate(count.index):
            fig.add_subplot(xnum,ynum,k+1).plot(clusters_dict[label]+0.5,alpha = 0.06)
#             plt.subplot(xnum,ynum,k+1).plot(clusters_dict[label].mean(1),'r')
            plt.plot(clusters_dict[label].mean(1)+0.5,'r')
    return clusters_dict

def save_dict(cluster_dicts, file_name):
    with open(file_name,'wb+') as f:
        pickle.dump(clusters_dict,f)
        
def normalized(load):
    peak = load.max(axis=1)[:, None]
    trough = load.min(axis=1)[:, None]
    diff = peak - trough
    diff[diff == 0.] = 1.
    normalized = (load - trough) / diff
    return normalized

def resample_normalization(data,granularity = '1min'):
    data.index = pd.date_range('2016-01-01-00:00:00','2016-01-01-23:59:59',freq = 'min')
    data = data.resample(granularity,label = 'left').mean()
    data = (data - data.min(0))/(data.max(0) - data.min(0)) - 0.5
    return data

In [ ]:
import pickle
def labeled_plot(X,plot_data,k=31,xnum=4,ynum=8,silent = True):
    '''
    k should <= xnum*ynum
    '''
    X = X.reset_index(drop = True)
    kmmodel = KMeans(k)
    kmmodel.fit(X.T)
    clusters_dict = {label:0 for label in set(kmmodel.labels_)}
    for i in kmmodel.labels_:
        clusters_dict[i] += 1
    count = pd.DataFrame(clusters_dict,index = ['count']).T.sort_values(by = 'count',ascending = False)
    clusters_dict = {label:0 for label in count.index}
    for label in count.index:
        clusters_dict[label] = plot_data.iloc[:,kmmodel.labels_ == label]
    if not silent:
        fig = plt.figure(figsize=(20,8))
        for k,label in enumerate(count.index):
            fig.add_subplot(xnum,ynum,k+1).plot(clusters_dict[label]+0.5,alpha = 0.3)
#             plt.subplot(xnum,ynum,k+1).plot(clusters_dict[label].mean(1),'r')
            plt.plot(clusters_dict[label].mean(1)+0.5,'r')
    return clusters_dict

def save_dict(cluster_dicts, file_name):
    with open(file_name,'wb+') as f:
        pickle.dump(clusters_dict,f)
        
def normalized(load):
    peak = load.max(axis=0)
    trough = load.min(axis=0)
    diff = peak - trough
    diff[diff == 0.] = 1.
    normalized = (load - trough) / diff
    return normalized

def resample_normalization(data,granularity = '1min'):
    data.index = pd.date_range('2016-01-01-00:00:00','2016-01-01-23:59:59',freq = 'min')
    data = data.resample(granularity,label = 'left').mean()
    data = normalized(data) - 0.5
    return data

In [ ]:
elbow_plot(resample_normalization(daily_pattern,'1min'),n_clusters = np.arange(1,40,2))

In [ ]:
plt.plot(resample_normalization(daily_pattern,'1min').iloc[:,0])

In [ ]:
elbow_plot(resample_normalization(daily_pattern,'1min'),n_clusters = np.arange(1,40,2))

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'1min'),5,2,3,silent = False)
for val in clusters_dict.values():
    print(val.shape[1])

In [ ]:
# granularity 1min 31 clusters
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'1min'),6,2,3,silent = False)

In [ ]:
# granularity 1min 31 clusters
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'1min'),resample_normalization(daily_pattern,'1min'), 7, 2, 4, silent=False)

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'1min'),12,3,4,silent = False)

In [ ]:
elbow_plot(resample_normalization(daily_pattern,'10min'),n_clusters = np.arange(1,40,2))

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'10min'),31,4,8,silent = False)

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'10min'),12,3,4,silent = False)

In [ ]:
elbow_plot(resample_normalization(daily_pattern,'30min'),n_clusters = np.arange(1,40,2))

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'30min'),5,2,3,silent=False)

In [ ]:
clusters_dict = labeled_plot(resample_normalization(daily_pattern,'30min'),12,3,4,silent=False)

In [ ]:
import pickle
with open('kmeans_granuity_1min.pkg','wb+') as f:
    pickle.dump(clusters_dict,f)

In [ ]:
daily_pattern.shape

In [ ]:
# kmeans with pca
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
latent = PCA(n_components = 16).fit_transform(resample_normalization(daily_pattern,'1min').T)
latent = pd.DataFrame(latent).T
cluster_dict = labeled_plot(latent,plot_data=resample_normalization(daily_pattern,'1min'),k=7,xnum=2,ynum=4,silent = False)




In [ ]:
daily_pattern.max(0)

In [ ]:
resample_normalization(daily_pattern)

In [ ]:
latent

In [ ]:
sigma = [0.00, 0.05, 0.10, 0.25, 0.35, 0.60, 0.73, 0.80]
kmeans = [0.614, 0.563, 0.433, 0.331, 0.094, 0.00, 0.00, 0.00]
ae = [1, 1, 1, 0.999, 0.999, 1, 0.985, 0]
for i in range(1,len(sigma)+1):
    fig,ax=plt.subplots()
    plt.rcParams['figure.figsize'] =(13.02,7.84)
    plt.plot(list(range(len(sigma[:i]))),np.array(ae[:i])*100,'x-',c='#029386',linewidth=3,markeredgewidth=5,markersize=10,label = 'Auto Encoder K-means')
    plt.plot(list(range(len(sigma[:i]))),np.array(kmeans[:i])*100,'x-',c='#fc5a50',linewidth=3,markeredgewidth=5,markersize=10,label = 'Naive K-means')
    
    plt.legend(fontsize=15,loc='lower left')
    plt.xticks(list(range(len(sigma))),sigma)
    plt.yticks([p for p in range(0, 101, 20)],[f'{p}%' for p in range(0, 101, 20)])
    plt.xlabel('$\sigma$',fontsize=20)
#     plt.ylabel('Success Rate',fontsize=20)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.savefig('graphics\\'+str(i)+'.png',format='png',dpi=600)

In [ ]:
840/317